### Toronto Segmenting and Clustering Notebook¶
Parsing attempt on a wikipedia page with BeautifulSoup. The target will be Toronto Neighborhood Wikipage as part of the IBM Professional Data Science Capstone week 3 assignment.

### Download and Clean Dataset from Wiki with BeautifulSoup¶
Use BeautifulSoup to parse neighborhood data from Toronto Neighborhood Wikipage.

Import required libraries

In [13]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd


# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#### Get html content to parse

In [14]:
# Get html with requests
page_link = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html_doc = requests.get(page_link, timeout=5)

# Parse html to beautiful soup object
soup = BeautifulSoup(html_doc.content, 'html.parser')

#### Extract table content from soup into list

In [15]:
table= []
for i in soup.table.tbody.find_all('td'):
    table.append(i.text.strip())

# Preview list
table[:12]

['M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village']

#### Split list into data frame¶

In [16]:
d = {'PostalCode': table[0::3], 'Borough': table[1::3], 'Neighborhood': table[2::3]}
df = pd.DataFrame(d)

df.head(10)

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


### Clean 'Not Assigned' values from Borough and Neighborhood¶

In [17]:
# Drop "Not Assigned" Borough
df = df[df.Borough != 'Not assigned']

# Replace "Not Assigned" Neighborhood with Borough value
df['Neighborhood'] = df.apply(lambda x: x['Borough'] if x['Neighborhood']=='Not assigned' else x['Neighborhood'], axis=1)
df.head(10)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


#### Group by PostalCode to concatenate Neighborhood entries

In [18]:
# Group by PostalCode to concatenate Neighborhood
df_postalgroup = df.groupby('PostalCode')

# Concatenate Neighborhood entries
y = df_postalgroup['Neighborhood'].apply(', '.join)
y = pd.DataFrame(y)

# Remove duplicate and merge
k = df[['PostalCode','Borough']].drop_duplicates(subset ="PostalCode")
df_torontogrouped = pd.merge(k,y,on = 'PostalCode', how = 'left')

df_torontogrouped.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [19]:
# Print shape
df_torontogrouped.shape

(103, 3)

### Populate Coordinates
Assign coordinates to the cleaned dataframe

#### Load geospatial coordinates

In [20]:
geodf = pd.read_csv("Geospatial_Coordinates.csv")
geodf.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


#### Join coordinates by postal code

In [21]:
geodf.rename(columns = {'Postal Code' : 'PostalCode'}, inplace = True)
df_coor = pd.merge(df_torontogrouped,geodf,on = 'PostalCode', how = 'left')
df_coor.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


### Analyze and Cluster Toronto Neighborhoods
#### Obtain coordinate of Toronto for Folium

In [22]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Drop everything except Toronto neighborhoods

In [24]:
toronto_data = df_coor[df_coor.Borough.str.contains("Toronto")].reset_index(drop = True)
toronto_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


In [25]:
toronto_data.loc[0, 'Neighborhood']

neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.6542599, -79.3606359.


#### Specify Credentials

In [30]:
CLIENT_ID = 'DT522EPPGQAZA2X3PPD2BSREECTFRCWSGVRLQCQ25QZULEQ4' # your Foursquare ID
CLIENT_SECRET = 'ET34M2HWMIMJMG4M5KYF2FQQNVQ01JORN0WTITM1NYVOQTUJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: DT522EPPGQAZA2X3PPD2BSREECTFRCWSGVRLQCQ25QZULEQ4
CLIENT_SECRET:ET34M2HWMIMJMG4M5KYF2FQQNVQ01JORN0WTITM1NYVOQTUJ


#### Construct Api calls and request json

In [31]:
# Construct URL
radius = 500
LIMIT = 100
# Constructed API calls
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
# Request JSON
results = requests.get(url).json()
results




{'meta': {'code': 200, 'requestId': '5f9bcf77a97868185adcdf79'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 90,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

In [32]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

#### import json and find nearby venues

In [33]:
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
3,Japango,Sushi Restaurant,43.655268,-79.385165
4,Indigo,Bookstore,43.653515,-79.380696


#### Use function from last exercise to get nearby venues of the neighborhood

In [40]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [41]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

90 venues were returned by Foursquare.


#### Get all nearby venues from each neighborhoods

In [42]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport


#### Check venue dataframe shape

In [43]:
print(toronto_venues.shape)
toronto_venues.head()

(3510, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Downtown Toronto,43.653232,-79.385296,Neighborhood
1,"Regent Park, Harbourfront",43.65426,-79.360636,Nathan Phillips Square,43.652270,-79.383516,Plaza
2,"Regent Park, Harbourfront",43.65426,-79.360636,Eggspectation Bell Trinity Square,43.653144,-79.381980,Breakfast Spot
3,"Regent Park, Harbourfront",43.65426,-79.360636,Japango,43.655268,-79.385165,Sushi Restaurant
4,"Regent Park, Harbourfront",43.65426,-79.360636,Indigo,43.653515,-79.380696,Bookstore


#### Group by count to check

In [44]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,90,90,90,90,90,90
"Brockton, Parkdale Village, Exhibition Place",90,90,90,90,90,90
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",90,90,90,90,90,90
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",90,90,90,90,90,90
Central Bay Street,90,90,90,90,90,90
Christie,90,90,90,90,90,90
Church and Wellesley,90,90,90,90,90,90
"Commerce Court, Victoria Hotel",90,90,90,90,90,90
Davisville,90,90,90,90,90,90


In [45]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 63 uniques categories.


#### Prepare for one hot encoding

In [46]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Neighborhood'], inplace=True, axis = 1)
#toronto_onehot['Neighborhood']
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 
toronto_onehot.columns
# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + toronto_onehot.columns[:-1].tolist()
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,...,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
toronto_onehot.shape

(3510, 63)

### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [48]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,American Restaurant,Art Museum,Bakery,Bank,Bar,Bookstore,Breakfast Spot,Bubble Tea Shop,Café,...,Sushi Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Women's Store
0,Berczy Park,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
1,"Brockton, Parkdale Village, Exhibition Place",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
2,"Business reply mail Processing Centre, South C...",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
4,Central Bay Street,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
5,Christie,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
6,Church and Wellesley,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
7,"Commerce Court, Victoria Hotel",0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
8,Davisville,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111
9,Davisville North,0.022222,0.011111,0.011111,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,...,0.022222,0.011111,0.011111,0.022222,0.022222,0.011111,0.011111,0.011111,0.011111,0.011111


#### Confirm new shape

In [49]:
toronto_grouped.shape

(39, 63)

#### print each neighborhood along with the top 5 most common venues

In [50]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Brockton, Parkdale Village, Exhibition Place----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2            Hotel  0.03
3  Thai Restaurant  0.02
4       Restaurant  0.02


----Central Bay Street----
             venue  freq
0      Coffee Shop  0.08
1   Clothing Store  0.07
2        

### Put  into a pandas dataframe


In [51]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Display top 10 venues for each neighborhood

In [52]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
1,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
3,"CN Tower, King and Spadina, Railway Lands, Har...",Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
4,Central Bay Street,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office


### Cluster Neighborhoods
#### Run k-means to cluster the neighborhood into 2 clusters.

In [57]:
# set number of clusters
kclusters = 2

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:37]

C:\Users\jisikhueme002\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\cluster\k_means_.py:972: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return_n_iter=True)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

#### Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [58]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted.astype({'Cluster Labels': 'int64'})

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how='right')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Clothing Store,Hotel,Plaza,Seafood Restaurant,Electronics Store,Diner,Cosmetics Shop,New American Restaurant,Office


#### Visualize resulting clusters

In [59]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters